In [1]:
!nvidia-smi
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

/bin/bash: nvidia-smi: command not found
--2022-02-13 10:03:16--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-02-13 10:03:16--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-02-13 10:03:16--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [applicatio

In [3]:
import pandas as pd
import numpy as np
from collections import Counter
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
train_df = pd.read_csv('mami_training.csv', sep='\t', lineterminator='\r')
test_df = pd.read_csv('Test.csv', sep='\t', lineterminator='\n')
test_labels = pd.read_csv('test_labels.txt',header=None, sep='\t', lineterminator='\n')
X = train_df['Text Transcription'].copy()
for i in range(len(X)):
  X[i] = X[i].lower()
X_test_model = test_df['Text Transcription'].to_numpy()
for i in range(len(X_test_model)):
  X_test_model[i] = X_test_model[i].lower()
Y = train_df['misogynous']
X_train_model = X[:].to_numpy()
Y_train_model = Y[:].to_numpy()
print(X_train_model.shape, Y_train_model.shape)
print(Y_train_model.sum())

(10000,) (10000,)
5000


In [4]:
embeddings_index = {}

f = open('glove.6B.200d.txt')
for line in f:
    values = line.split()
    word = value = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' %len(embeddings_index))

Found 400000 word vectors.


In [5]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence  import pad_sequences

tokenizer_obj = Tokenizer()
total_memes =  train_df['Text Transcription']
tokenizer_obj.fit_on_texts(total_memes)

max_length = 50 #max([len(s.split()) for s in total_memes])
vocab_size  = len(tokenizer_obj.word_index) + 1
X_train_tokens = tokenizer_obj.texts_to_sequences(X_train_model)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test_model)
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length)
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length)
print(vocab_size)

20693


In [6]:
EMBEDDING_DIM = 200
word_index = tokenizer_obj.word_index
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

def average_glove(df):
  real_X = np.zeros((len(df) , EMBEDDING_DIM))
  for i in range(len(df)):
    num = 0
    for word in df[i]:
      num += 1
      real_X[i] += embedding_matrix[int(word)]
    if num!=0:
      real_X[i] = real_X[i]/num
  return real_X

X_train_pad_2_vec = average_glove(X_train_pad)
X_test_pad_2_vec = average_glove(X_test_pad)
print(X_test_pad_2_vec.shape,X_train_pad_2_vec.shape )

(1000, 200) (10000, 200)


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
Y_test_model = test_labels[1].to_numpy()
Y_train_model = train_df['misogynous'].to_numpy()
clf = RandomForestClassifier(n_jobs=5, bootstrap=True,oob_score=True, n_estimators=10, max_features=None)
clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Misogynous Prediction ########")
print("\nTraining Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = clf.predict(X_test_pad_2_vec)
print("\nSubmission Score ")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskA = f1_score(Y_test_model, y_pred, average='macro')
subtaskB = f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:564: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  UserWarning,


######## Misogynous Prediction ########

Training Dataset 
[[3931   32]
 [  83 3954]] 
 F1 score -  0.9856248811806585

Validation Dataset 
[[715 322]
 [432 531]] 
 F1 score -  0.6197818334382212

Submission Score 
[[284 216]
 [195 305]] 
 F1 score -  0.5888186690330436


In [8]:
Y_train_model = train_df['shaming'].to_numpy()
Y_test_model = test_labels[2].to_numpy()
clf = RandomForestClassifier(n_jobs=5, bootstrap=True,oob_score=True, n_estimators=10, max_features=None)
clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Shaming Prediction ########")
print("\nTraining Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = clf.predict(X_test_pad_2_vec)
print("\n######## Shaming Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:564: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  UserWarning,


######## Shaming Prediction ########

Training Dataset 
[[6937    7]
 [ 151  905]] 
 F1 score -  0.9542277349797443

Validation Dataset 
[[1749   33]
 [ 209    9]] 
 F1 score -  0.502262443438914

######## Shaming Prediction ########
[[836  18]
 [142   4]] 
 F1 score -  0.4801414015387815


In [9]:
Y_train_model = train_df['stereotype'].to_numpy()
Y_test_model = test_labels[3].to_numpy()
clf = RandomForestClassifier(n_jobs=5, bootstrap=True,oob_score=True, n_estimators=10, max_features=None)
clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Stereotype Prediction ########")
print("\nTraining Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = clf.predict(X_test_pad_2_vec)
print("\n######## Stereotype Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:564: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  UserWarning,


######## Stereotype Prediction ########

Training Dataset 
[[5567   17]
 [ 170 2246]] 
 F1 score -  0.9717581099485098

Validation Dataset 
[[1444  162]
 [ 308   86]] 
 F1 score -  0.5639742540712744

######## Stereotype Prediction ########
[[604  46]
 [281  69]] 
 F1 score -  0.5418724387937375


In [10]:
Y_train_model = train_df['objectification'].to_numpy()
Y_test_model = test_labels[4].to_numpy()
clf = RandomForestClassifier(n_jobs=5, bootstrap=True,oob_score=True, n_estimators=10, max_features=None)
clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Objectification Prediction ########")
print("\nTraining Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = clf.predict(X_test_pad_2_vec)
print("\n######## Objectification Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:564: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  UserWarning,


######## Objectification Prediction ########

Training Dataset 
[[6158    9]
 [ 167 1666]] 
 F1 score -  0.9678699727004303

Validation Dataset 
[[1547   84]
 [ 332   37]] 
 F1 score -  0.5162509448223734

######## Objectification Prediction ########
[[620  32]
 [323  25]] 
 F1 score -  0.45044312860404817


In [11]:
Y_train_model = train_df['violence'].to_numpy()
Y_test_model = test_labels[5].to_numpy()
clf = RandomForestClassifier(n_jobs=5, bootstrap=True,oob_score=True, n_estimators=10, max_features=None)
clf.fit(X_train_pad_2_vec[:8000], Y_train_model[:8000])
print("######## Violence Prediction ########")
print("\nTraining Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[:8000])
print(confusion_matrix(Y_train_model[:8000], y_pred),"\n F1 score - ", f1_score(Y_train_model[:8000], y_pred, average='macro'))
print("\nValidation Dataset ")
y_pred = clf.predict(X_train_pad_2_vec[8000:])
print(confusion_matrix(Y_train_model[8000:], y_pred),"\n F1 score - ", f1_score(Y_train_model[8000:], y_pred, average='macro'))
y_pred = clf.predict(X_test_pad_2_vec)
print("\n######## Violence Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:564: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  UserWarning,


######## Violence Prediction ########

Training Dataset 
[[7163    6]
 [ 140  691]] 
 F1 score -  0.947180907562247

Validation Dataset 
[[1859   19]
 [ 120    2]] 
 F1 score -  0.4959668280902401

######## Violence Prediction ########
[[846   1]
 [151   2]] 
 F1 score -  0.47160576227821344


In [12]:
print("Subtask A score -", 
      subtaskA, 
      "\n Subtask B score - ", 
      subtaskB/(test_labels[5].sum()+ test_labels[4].sum()+ test_labels[3].sum()+test_labels[2].sum()+test_labels[1].sum()))

Subtask A score - 0.5888186690330436 
 Subtask B score -  0.5230963414173463
